In [61]:
import pandas as pd
import numpy as np


from my_modules import my_functions as mybib
from my_modules import project_functions as pr

import os # for file system operations
import re # for regular expressions
from tqdm import tqdm # for progress bar

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
column_names = [i for i in range(0, 15)]

population_df = pd.DataFrame(columns=column_names)
display(population_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14


In [63]:
def transform_number_under_10(x):
    if x < 10:
        x = '0' + str(x)
        return x
    else:
        x = str(x)
        return x

In [64]:


# https://ecoagi.ai/de/topics/Python/python-get-all-files-in-directory

# creating a list of all files in the directory
list_xlsx_files = os.listdir('../data/input/population')
list_xlsx_files

['SB_A01-16-00_2022h02_BE.xlsx']

In [65]:




# Loop through all the files in the directory
for filename in tqdm(list_xlsx_files): 

    print(f'{filename}')
    
    # extract year and half_year from filename
    temp = re.search(r'(\d{4})h(\d{2})', filename)
    year, half_year = temp.groups()

    # create file path for file import 
    file_path = f'../data/input/population/{filename}'

    # import data from 'T2' sheet - checked in Excel 
    raw_data = pd.read_excel(file_path, 'T2', header = None)

    # checked with excel the dataset before -> after dropping rows with Nan values the needed information are extracted 
    final_rows = raw_data.dropna().index.tolist()
    pop_data = raw_data.iloc[final_rows]
        
    # Add 2 columns year and half_year
    pop_data.insert(0, 'year', year)
    pop_data.insert(1, 'half_year', half_year)
    
    # counter = counter + len(pop_data)
        
    population_df = pd.concat([population_df, pop_data], ignore_index=True , axis=0)

    # Convert columns to integer 
    columns_to_transform = population_df.columns[0:17]
    population_df[columns_to_transform] = population_df[columns_to_transform].astype(int)
    # print(population_df.dtypes)
    
    # Transform format for LOR values - (1 -> 01, 2 -> 02 ...)
    for col in [0,1,2,3]:
        population_df[col] = population_df[col].apply(lambda x: transform_number_under_10(x))
    


    # drop woman and asyl data columns 
    # transform 0-3 to LOR Code
    # Rearrange table columns    
    population_df['lor'] = population_df[0] + population_df[1] + population_df[2] + population_df[3]
    population_df['key'] = population_df['lor'] + '-' + population_df['year'].astype(str) + '-' +  population_df['half_year'].astype(str)
    
# Drop unnecessary columns 
population_df = population_df.drop(columns = [0, 1, 2, 3, 13, 14])

# Reorder and rename columns 
population_df = population_df.reindex(['key', 'year', 'half_year', 'lor', 4, 5, 6, 7, 8, 9, 10, 11, 12], axis = 1)
population_df.columns = ['key', 'year', 'half_year', 'lor', 'total_population', '-6', '6-15', '15-18', '18-27', '27-45', '45-55', '55-65', '65+']


# Save the final DataFrame to a pickle file 
population_df.to_pickle('../data/output/temp_analysis/total_population_dataset.pkl')
# population_df.to_excel('../data/output/total_population_dataset.xlsx', index = False)
# population_df.to_csv('../data/output/total_population_dataset.csv', index = False)

display(population_df.head(5))



  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

SB_A01-16-00_2022h02_BE.xlsx


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.74it/s]


,key,year,half_year,lor,total_population,-6,6-15,15-18,18-27,27-45,45-55,55-65,65+
0,01100101-2022-2,2022,2,01100101,3561,165,209,25,395,1271,375,403,718
1,01100102-2022-2,2022,2,01100102,2002,113,129,27,174,764,274,262,259
2,01100103-2022-2,2022,2,01100103,5662,282,326,102,741,1791,653,747,1020
3,01100104-2022-2,2022,2,01100104,5029,260,397,137,801,1886,601,554,393
4,01100205-2022-2,2022,2,01100205,3009,147,178,52,390,1122,331,319,470


In [66]:
# https://ecoagi.ai/de/topics/Python/python-get-all-files-in-directory



# Loop through all the files in the directory
for filename in tqdm(list_xlsx_files): 

    
    # extract year and half_year from filename
    temp = re.search(r'(\d{4})h(\d{2})', filename)
    year, half_year = temp.groups()

    # create file path for file import 
    file_path = f'../data/input/population/{filename}'

    # import data from 'T2' sheet - checked in Excel 
    raw_data = pd.read_excel(file_path, 'T2', header = None)

    # checked with excel the dataset before -> after dropping rows with Nan values the needed information are extracted 
    Nan_rows = raw_data.isnull().any(axis=1)
    Non_nan_rows = ~Nan_rows
    final_rows = Non_nan_rows[Non_nan_rows].index.tolist()
    pop_data = raw_data.iloc[final_rows]
        
    pop_data.insert(0, 'year', year)
    pop_data.insert(1, 'half_year', half_year)
    
    # counter = counter + len(pop_data)
        
    population_df = pd.concat([population_df, pop_data], ignore_index=True , axis=0)
    

    # display(population_df.head(1)) 
    # print(population_df.dtypes)
    # print(population_df.dtypes)
    columns_to_transform = population_df.columns[0:17]
    population_df[columns_to_transform] = population_df[columns_to_transform].astype(int)
    # print(population_df.dtypes)
    
    
    columns_to_transform = [0,1,2,3]
    # print(f'File {year}-{half_year} shape: {pop_data.shape} \n')
    for col in columns_to_transform:
        population_df[col] = population_df[col].apply(lambda x: transform_number_under_10(x))
    
    # display(population_df.head(1))
    # print(population_df.dtypes)
    

    # drop woman and asyl data columns 
    # transform 0-3 to LOR Code
    # Rearrange table columns    
    population_df['lor'] = population_df[0] + population_df[1] + population_df[2] + population_df[3]
    population_df['key'] = population_df['lor'] + '-' + population_df['year'].astype(str) + '-' +  population_df['half_year'].astype(str)
    

population_df = population_df.drop(columns = [0, 1, 2, 3, 13, 14])

population_df = population_df.reindex(['key', 'year', 'half_year', 'lor', 4, 5, 6, 7, 8, 9, 10, 11, 12], axis = 1)
population_df.columns = ['key', 'year', 'half_year', 'lor', 'total_population', '-6', '6-15', '15-18', '18-27', '27-45', '45-55', '55-65', '65+']



population_df.to_pickle('../data/output/temp_analysis/total_population_dataset.pkl')
# population_df.to_excel('../data/output/total_population_dataset.xlsx', index = False)
# population_df.to_csv('../data/output/total_population_dataset.csv', index = False)

display(population_df.head(5))
# print(population_df.columns)
# print(population_df.dtypes)


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


ValueError: invalid literal for int() with base 10: '01100101-2022-2'

In [ ]:
# https://ecoagi.ai/de/topics/Python/python-get-all-files-in-directory




# for filename in tqdm(list_xlsx_files): 

    
#     # extract year and half_year
#     temp = re.search(r'(\d{4})h(\d{2})', filename)
#     year, half_year = temp.groups()
#     # print(f'{year}, {half_year}')
    
#     # print(f'{filename}')
    

#     # create file path for file import 
#     file_path = f'../data/input/population/{filename}'

#     # import data 
#     raw_data = pd.read_excel(file_path, 'T2', header = None)

    
#     Nan_rows = raw_data.isnull().any(axis=1)
#     Non_nan_rows = ~Nan_rows
#     final_rows = Non_nan_rows[Non_nan_rows].index.tolist()
#     pop_data = raw_data.iloc[final_rows]
        
#     pop_data.insert(0, 'year', year)
#     pop_data.insert(1, 'half_year', half_year)
    
#     # counter = counter + len(pop_data)
        
#     population_df = pd.concat([population_df, pop_data], ignore_index=True , axis=0)
    

#     # display(population_df.head(1)) 
#     # print(population_df.dtypes)
#     # print(population_df.dtypes)
#     columns_to_transform = population_df.columns[0:17]
#     population_df[columns_to_transform] = population_df[columns_to_transform].astype(int)
#     # print(population_df.dtypes)
    
    
#     columns_to_transform = [0,1,2,3]
#     # print(f'File {year}-{half_year} shape: {pop_data.shape} \n')
#     for col in columns_to_transform:
#         population_df[col] = population_df[col].apply(lambda x: transform_number_under_10(x))
    
#     # display(population_df.head(1))
#     # print(population_df.dtypes)
    

#     # drop woman and asyl data columns 
#     # transform 0-3 to LOR Code
#     # Rearrange table columns    
#     population_df['lor'] = population_df[0] + population_df[1] + population_df[2] + population_df[3]
#     population_df['key'] = population_df['lor'] + '-' + population_df['year'].astype(str) + '-' +  population_df['half_year'].astype(str)
    

    
#     population_df.to_csv(f'../data/output/temp_population_data/population-{year}-{half_year}.csv', index = False)
#     # display(population_df.head(1))


# # [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 'year', 'half_year', ]



# population_df = population_df.drop(columns = [0, 1, 2, 3, 13, 14])

# population_df = population_df.reindex(['key', 'year', 'half_year', 'lor', 4, 5, 6, 7, 8, 9, 10, 11, 12], axis = 1)
# population_df.columns = ['key', 'year', 'half_year', 'lor', 'total_population', '-6', '6-15', '15-18', '18-27', '27-45', '45-55', '55-65', '65+']



# population_df.to_csv('../data/output/total_population_dataset.csv', index = False)
# population_df.to_pickle('../data/output/total_population_dataset.pkl')
# population_df.to_excel('../data/output/total_population_dataset.xlsx', index = False)

# display(population_df.head(5))
# print(population_df.columns)
# print(population_df.dtypes)
